## Exporting yolo v8 models to CoreML

It should use [Ultralytics](https://github.com/ultralytics/ultralytics) to convert a Pytorch model to CoreML and use it in an [iOS app demo](https://github.com/ultralytics/yolo-ios-app) made by them.


### Download ultralytics

if not present

In [1]:
DETECTOR_DIR = "third_party/ultralytics/"
!git -C {DETECTOR_DIR} pull || git clone https://github.com/ultralytics/ultralytics {DETECTOR_DIR}

Updating 19a2d0a4..997f2c92
error: Your local changes to the following files would be overwritten by merge:
	ultralytics/nn/tasks.py
Please commit your changes or stash them before you merge.
Aborting
fatal: destination path 'third_party/ultralytics' already exists and is not an empty directory.


### Download ios-demo-app

In [2]:
DEMO_APP_DIR = "third_party/ultralytics-yolo-ios-app/"
!git -C {DEMO_APP_DIR} pull || git clone https://github.com/ultralytics/yolo-ios-app {DEMO_APP_DIR}

fatal: cannot change to 'third_party/ultralytics-yolo-ios-app/': No such file or directory
Cloning into 'third_party/ultralytics-yolo-ios-app'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 119 (delta 37), reused 20 (delta 20), pack-reused 67
Receiving objects: 100% (119/119), 122.72 KiB | 294.00 KiB/s, done.
Resolving deltas: 100% (42/42), done.


### Export the model using ultralytics

It will download the model if necessary.

It's a good idea to use the ultralytics pip package, because you need to set the correct version of numpy, pytorch etc. Also, I got erros using Python versions above 3.10, the following was tested with Python 3.10.14.

In [7]:
!pip install ultralytics

In [1]:
import sys, os
sys.path.append("third_party/ultralytics")
from ultralytics import YOLO

model_name = "yolov8n"

# Load a model
model = YOLO("yolov8n.pt")  # load an official model

# Export the model
model.export(format="coreml",  int8=True, nms=True, imgsz=[640, 384])

# run using the ultralytics library
# Load a model
# model = YOLO("yolov8n.mlpackage", task="detect")  # pretrained YOLOv8n model

# results = model(["third_party/ultralytics/ultralytics/assets/zidane.jpg"])


Ultralytics YOLOv8.2.52 🚀 Python-3.10.14 torch-2.3.1 CPU (Apple M3)
WARNING ⚠️ INT8 export requires a missing 'data' arg for calibration. Using default 'data=coco8.yaml'.
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 384) BCHW and output shape(s) (1, 84, 5040) (6.2 MB)


scikit-learn version 1.5.1 is not supported. Minimum required version: 0.17. Maximum required version: 1.1.2. Disabling scikit-learn conversion API.
Torch version 2.3.1 has not been tested with coremltools. You may run into unexpected errors. Torch 2.2.0 is the most recent version that has been tested.



CoreML: starting export with coremltools 7.2...


Tuple detected at graph output. This will be flattened in the converted model.
Running compression pass palettize_weights: 100%|██████████| 129/129 [00:26<00:00,  4.79 ops/s]
Running MIL frontend_milinternal pipeline: 0 passes [00:00, ? passes/s]
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 198.93 passes/s]


CoreML Pipeline: starting pipeline with coremltools 7.2...
CoreML Pipeline: pipeline success
CoreML: export success ✅ 42.4s, saved as 'yolov8n.mlpackage' (3.2 MB)

Export complete (43.0s)
Results saved to /Users/gustavofuhr/projects/object_detection_ios_comprehensive
Predict:         yolo predict task=detect model=yolov8n.mlpackage imgsz=640,384 int8 
Validate:        yolo val task=detect model=yolov8n.mlpackage imgsz=640,384 data=coco.yaml int8 WARNING ⚠️ non-PyTorch val requires square images, 'imgsz=[640, 384]' will not work. Use export 'imgsz=640' if val is required.
Visualize:       https://netron.app


'yolov8n.mlpackage'

In [2]:
# check the size of the ml package
!du -sh yolov8n.mlpackage

3.2M	yolov8n.mlpackage
